In [13]:
narrator_dir="/home/ansary/work/hadith/narrator_data"
hadith_book_dir="/home/ansary/work/hadith/hadith_data_bookwise"
save_dir="/home/ansary/work/hadith/narrator_data_bookwise"

In [10]:
import os 
from glob import glob
from tqdm import  tqdm
import pandas as pd 
tqdm.pandas()
import json

books=os.listdir(hadith_book_dir)

df=[]

for book in books:
    hadiths=[f for f in glob(os.path.join(hadith_book_dir,book,"*.json"))]
    for hadith in tqdm(hadiths):
        with open(hadith,"r") as f:
            data=json.load(f)
        hadith_id=os.path.basename(hadith).split(".")[0]
        narrators=[d["id"] for d in data["_source"]["narrators"]]
        # for nid in narrators:
        #     if int(nid)<=0: print(hadith_id,nid)
        df.append({"book":book,"hadith_id":hadith_id,"narrators":narrators})
df=pd.DataFrame(df)
df



  0%|          | 0/5367 [00:00<?, ?it/s]

100%|██████████| 5691/5691 [00:00<00:00, 6324.77it/s]


,book,hadith_id,narrators
0,صحيح مسلم,158-1728,"[2346, 5147, 8621, 7272, 2542, 7570]"
1,صحيح مسلم,158-2152,"[970, 6861, 4493, 4232, 3436, 2158, 6539, 5471..."
2,صحيح مسلم,158-1310,"[3036, 927, 2122, 7673, 902, 5715, 4049]"
3,صحيح مسلم,158-1961,"[5435, 7561, 6622, 4861, 4049]"
4,صحيح مسلم,158-49,"[3036, 6021, 5702, 8441, 4396]"
...,...,...,...
82591,سنن النسائى الصغرى,319-1825,"[6171, 4207, 3281, 2550, 2625, 3281, 6458, 299..."
82592,سنن النسائى الصغرى,319-4597,"[8572, 3430, 8042, 5703, 4883]"
82593,سنن النسائى الصغرى,319-5601,"[3719, 3719, 4716, 4716, 7633, 8621, 7272, 727..."
82594,سنن النسائى الصغرى,319-4428,"[6460, 3443, 4797, 4967]"


In [11]:
# Explode the narrators column to get one narrator per row
df_exploded = df.explode("narrators")

# Rename the column for clarity
df_exploded.rename(columns={"narrators": "narrator_id"}, inplace=True)

# Group by book and narrator_id, then aggregate hadith_id into a list
result = df_exploded.groupby(["book", "narrator_id"])["hadith_id"].apply(list).reset_index()

# Rename columns
result.rename(columns={"hadith_id": "narrated_hadiths"}, inplace=True)
result

,book,narrator_id,narrated_hadiths
0,المستدرك على الصحيحين,-1,"[594-6824, 594-6824, 594-6824, 594-6824, 594-6..."
1,المستدرك على الصحيحين,-2,"[594-6819, 594-6819, 594-6819, 594-6805, 594-6..."
2,المستدرك على الصحيحين,-3,[594-6805]
3,المستدرك على الصحيحين,-4,[594-6805]
4,المستدرك على الصحيحين,0,[594-3222]
...,...,...,...
37861,موطأ مالك رواية يحيى الليثي,8622,"[19-1320, 19-1580, 19-1584]"
37862,موطأ مالك رواية يحيى الليثي,93,[19-551]
37863,موطأ مالك رواية يحيى الليثي,952,"[19-1573, 19-1737, 19-1809, 19-1717, 19-1730, ..."
37864,موطأ مالك رواية يحيى الليثي,980,[19-147]


In [14]:
for book in result.book.unique():
    book_save=os.path.join(save_dir,book)
    os.makedirs(book_save,exist_ok=True)
    bdf=result.loc[result.book==book]
    bdf.reset_index(inplace=True,drop=True)
    for idx in tqdm(range(len(bdf))):
        nid=bdf.iloc[idx,1]
        hadiths=bdf.iloc[idx,-1]
        nar_json=os.path.join(narrator_dir,f"{nid}.json")
        with open(nar_json,"r") as f: base_data=json.load(f)
        base_data[nid]["narrated_hadith_ids"]=hadiths
        with open(os.path.join(book_save,f"{nid}.json") ,"w") as f: json.dump(base_data,f,indent= 2,ensure_ascii=False)

  0%|          | 0/6435 [00:00<?, ?it/s]

100%|██████████| 499/499 [00:00<00:00, 1076.18it/s]
